In [ ]:
# Im Terminal:
# 1) Folder erstellen:
# cd C:\Users\Engineer\"OneDrive - DataCraft GmbH"\Dokumente\DataCraft\"4.1. Vorbereitung Praxis"

# 2) Scrapy Projekt anlegen
# scrapy startproject Linkedin_Firmen_infos

# 3) in Projektverzeichnis navigieren:
# cd Linkedin_Firmen_infos

# 4) Spider erstellen
# scrapy genspider Linkedin_Firmen_Spider linkedin.com

In [11]:
# code für Crawler
#import
import json
import scrapy


# Klassen definieren
class Linkedin_Firmen_Spider(scrapy.Spider):
    name = "linkedin_company_profile"

    # hier kommen alle public linkedin pages rein von der cryptofundraising seite, die mit nem andern Crawler abgegriffen werden
    public_profile_webpages= [
        'https://www.linkedin.com/company/kryptoskatt/about/',
        ""
        ]


    def start_requests(self):
        
        public_profile_index_tracker = 0

        start_url = self.public_profile_webpages[public_profile_index_tracker]

        yield scrapy.Request(url=start_url, callback=self.parse_response, meta={'public_profile_index_tracker': public_profile_index_tracker})


    def parse_response(self, response):
        public_profile_index_tracker = response.meta['public_profile_index_tracker']
        print('***************')
        print('****** Scraping page ' + str(public_profile_index_tracker+1) + ' of ' + str(len(self.public_profile_webpages)))
        print('***************')

        Firmen_items = {}

        # Frage ob Deutsch oder Englisch besser ist
        # Vermutlich besser auf Englisch? Oder muss ich es aus Deutsch ändern, weil ich alles auf Deutsch in Linkedin anschaue?        
        Firmen_items['name'] = response.css('.top-card-layout__entity-info h1::text').get(default='not-found').strip()
        Firmen_items['summary'] = response.css('.top-card-layout__entity-info h4 span::text').get(default='not-found').strip()

        try:
            ## alle Firmen infos
            Firmen_details = response.css('.core-section-container__content .mb-2')

            # Branchen
            Firma_Branchentyp = Firmen_details[1].css('.text-md::text').getall()
            Firmen_items['industry'] = Firma_Branchentyp[1].strip()

            # Größe_der_Firma
            Größe_der_Firma = Firmen_details[2].css('.text-md::text').getall()
            # Wäre jetzt die Frage, ob es dann englisch oder Deutsch sein muss?
            Firmen_items['size'] = Größe_der_Firma[1].strip()

            #Gründungsdatum
            Gründung_der_Firma = Firmen_details[5].css('.text-md::text').getall()
            Firmen_items['founded'] = Gründung_der_Firma[1].strip()
        except IndexError:
            print("Error: Firme wurde ausgelassen - Einige Infos fehlen!")

        yield Firmen_items
        

        public_profile_index_tracker = public_profile_index_tracker + 1

        if public_profile_index_tracker <= (len(self.public_profile_webpages)-1):
            next_url = self.public_profile_webpages[public_profile_index_tracker]

            yield scrapy.Request(url=next_url, callback=self.parse_response, meta={'public_profile_index_tracker': public_profile_index_tracker})
    

In [8]:
# Crawler laufen lassen:
# über terminal: scrapy crawl linkedin_company_profile

SyntaxError: invalid syntax (2674254098.py, line 2)

In [3]:
# nicht geblockt werden => weitere packages notwendig:
# pip install scrapeops-scrapy

In [ ]:
# Eigene ScrapeOps API key hinzufügen
SCRAPEOPS_API_KEY = 'UNSER_API_Key'


# Hinzufügen von Erweiterungen im ScrapeOps Extension
EXTENSIONS = {
'scrapeops_scrapy.extension.ScrapeOpsMonitor': 500, 
}


# Update der Downloader Middlewares
DOWNLOADER_MIDDLEWARES = {
'scrapeops_scrapy.middleware.retry.RetryMiddleware': 550,
'scrapy.downloadermiddlewares.retry.RetryMiddleware': None,
}
